## Programme pour creer la table teams

In [51]:
# Import des librairies
import pandas as pd
import requests
import pandas_gbq
import getpass
from google.oauth2 import service_account
pd.set_option('display.max_columns', None)

In [52]:
# On va chercher les equipes
url = 'https://statsapi.web.nhl.com/api/v1/teams'
r = requests.get(url)
json_data = r.json()
teams = pd.json_normalize(json_data['teams'], sep = "_")

In [53]:
# On garde seulement les colonnes interessantes
teams = teams[['id', 'abbreviation', 'teamName', 'locationName','officialSiteUrl', 'active', 'venue_name', 'venue_city']]

# On renomme pour avoir des noms coherents avec ce qui se trouve dans la BD
teams = teams.rename(columns={'id': 'team_id',
                              'abbreviation': 'abbreviation',
                              'teamName': 'team_name',
                              'locationName': 'team_city',
                              'officialSiteUrl': 'website_url',
                              'active': 'is_active',
                              'venue_name': 'venue_name',
                              'venue_city': 'venue_city'
                             })

In [54]:
teams.head()

,team_id,abbreviation,team_name,team_city,website_url,is_active,venue_name,venue_city
0,1,NJD,Devils,New Jersey,http://www.newjerseydevils.com/,True,Prudential Center,Newark
1,2,NYI,Islanders,New York,http://www.newyorkislanders.com/,True,Nassau Veterans Memorial Coliseum,Uniondale
2,3,NYR,Rangers,New York,http://www.newyorkrangers.com/,True,Madison Square Garden,New York
3,4,PHI,Flyers,Philadelphia,http://www.philadelphiaflyers.com/,True,Wells Fargo Center,Philadelphia
4,5,PIT,Penguins,Pittsburgh,http://pittsburghpenguins.com/,True,PPG Paints Arena,Pittsburgh


In [55]:
def get_keys():
    '''
    This function will return the path to Google Keys for each user.
    '''
    if getpass.getuser()=='antoinetl':
        credential_keys = '/Users/antoinetl/Documents/code/Google Keys/My First Project-4938b2ab0dc6.json'
    elif getpass.getuser()=='philippejacques':
        credential_keys = '/Users/philippejacques/Desktop/Projet/HockeyPrediction/Google Keys/hockey-prediction-qc-b427ad188c20.json'
    return credential_keys

In [56]:
# Connexion GBQ
credential_keys = get_keys()

# Connexion GBQ
credentials = service_account.Credentials.from_service_account_file(
    credential_keys,
    )
pandas_gbq.context.credentials = credentials

In [57]:
# GCP project ID.
project_id = "hockey-prediction-qc"

# table_id that corresponds to the name of the table.
table_id = 'hockey_prediction_qc.teams'

# Insert into GCP
pandas_gbq.to_gbq(teams,
                  table_id,
                  project_id=project_id,
                  if_exists='replace')

1it [00:03,  3.99s/it]
